**Regresión polinomial.**

*Uso de la validación cruzada.*

Hello, I'm Go1234550

Realizaremos una regresión lineal múltiple para predecir el precio de una casa haciendo uso de las siguientes variables:

    - sqft_living(pies cuadrados de área habitable): Indica el área total, en pies cuadrados, de la superficie habitable de una vivienda.

    - bedrooms: Numero de cuartos que cuenta la vivienda.
    
    - bathrooms: Cantidad de baños en la vivienda.

Se consideran estas variables ya que son las que tienen mayor influencia para la determinación del precio de una propiedad.